In [78]:
import pandas as pd


# pd.set_option('precision',50)
# pd.set_option('chop_threshold', .00000000005)


df = pd.read_pickle("data/vision_baseline_graph_data.pkl")

df.describe()


,timedelta,root_distance_0,current,beam_node,has_followed_path,has_follow_path,conversation_id,reply_distance_1,reply_distance_2,reply_distance_3,...,root_distance_14,root_distance_15,root_distance_16,root_distance_17,root_distance_18,root_distance_19,root_distance_20,root_distance_21,root_distance_22,root_distance_23
count,8.098230e+05,809823.000000,8.098230e+05,8.098230e+05,809823.0,809823.0,8.098230e+05,809823.000000,809823.000000,809823.000000,...,809823.000000,809823.000000,809823.000000,809823.000000,809823.000000,809823.000000,809823.000000,809823.000000,809823.000000,809823.000000
mean,5.369646e+04,0.038011,1.481440e+18,1.481198e+18,0.0,0.0,1.480454e+18,0.012676,0.002759,0.000710,...,0.000254,0.000278,0.000184,0.000111,0.000081,0.000073,0.000065,0.000095,0.000081,0.000073
std,8.978862e+05,0.191222,2.648856e+17,2.641527e+17,0.0,0.0,2.624457e+17,0.111870,0.037041,0.013307,...,0.015947,0.016666,0.013563,0.010541,0.009027,0.008535,0.008090,0.009751,0.009027,0.008535
min,1.000000e-06,0.000000,2.151180e+05,2.151180e+05,0.0,0.0,1.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.053000e+03,0.000000,1.511883e+18,1.511768e+18,0.0,0.0,1.511628e+18,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.869000e+03,0.000000,1.524812e+18,1.524722e+18,0.0,0.0,1.524646e+18,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.365750e+04,0.000000,1.543267e+18,1.543217e+18,0.0,0.0,1.542981e+18,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.103033e+08,1.000000,7.694580e+18,7.694580e+18,0.0,0.0,1.544234e+18,1.000000,0.500000,0.250000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [79]:
# sum all reply columns

reply_filter_col = [col for col in df if col.startswith('reply')]
root_distance_filter_col = [col for col in df if col.startswith('root')]
reply_filter_col

['reply_distance_1',
 'reply_distance_2',
 'reply_distance_3',
 'reply_distance_4',
 'reply_distance_5',
 'reply_distance_6',
 'reply_distance_7',
 'reply_distance_8',
 'reply_distance_9',
 'reply_distance_10',
 'reply_distance_11',
 'reply_distance_12',
 'reply_distance_13',
 'reply_distance_14',
 'reply_distance_15',
 'reply_distance_16',
 'reply_distance_17',
 'reply_distance_18',
 'reply_distance_19',
 'reply_distance_20',
 'reply_distance_21',
 'reply_distance_22',
 'reply_distance_23']

In [80]:
reply_columns = df[reply_filter_col]
root_distance_columns= df[root_distance_filter_col]
reply_columns

,reply_distance_1,reply_distance_2,reply_distance_3,reply_distance_4,reply_distance_5,reply_distance_6,reply_distance_7,reply_distance_8,reply_distance_9,reply_distance_10,...,reply_distance_14,reply_distance_15,reply_distance_16,reply_distance_17,reply_distance_18,reply_distance_19,reply_distance_20,reply_distance_21,reply_distance_22,reply_distance_23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
809819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
809820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
809821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
reply_cs = reply_columns.sum(axis=1)
root_distance_cs = root_distance_columns.sum(axis=1)

rcs_not_null = [i for i in reply_cs.tolist() if i != 0]
#rcs_not_null

In [82]:
root_reply_combined = (root_distance_cs + reply_cs)
root_reply_combined = (root_reply_combined - root_reply_combined.min()) / (root_reply_combined.max() - root_reply_combined.min())
combined = [i for i in root_reply_combined.tolist() if i != 0]

In [83]:
#
df = df.assign(root_reply_combined=root_reply_combined)
df

,timedelta,root_distance_0,current,beam_node,has_followed_path,has_follow_path,platform,conversation_id,reply_distance_1,reply_distance_2,...,root_distance_15,root_distance_16,root_distance_17,root_distance_18,root_distance_19,root_distance_20,root_distance_21,root_distance_22,root_distance_23,root_reply_combined
0,2189.0,0,1511697071241416709,1511687891377868800,0,0,twitter,1511667824032428039,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2689.0,0,1511697071241416709,1511685795056340994,0,0,twitter,1511667824032428039,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,776.0,0,1511697071241416709,1511693816465612806,0,0,twitter,1511667824032428039,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6559.0,0,1511697071241416709,1511669562487541764,0,0,twitter,1511667824032428039,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6671.0,0,1511697071241416709,1511669091966275584,0,0,twitter,1511667824032428039,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809818,47015.0,0,1542589964856070144,1542392772342300672,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
809819,39754.0,0,1542589964856070144,1542423225153486851,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
809820,34536.0,0,1542589964856070144,1542445109698039808,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
809821,8943.0,0,1542589964856070144,1542552458227884033,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5


In [84]:
import numpy as np

df_data = df[["root_reply_combined", "conversation_id", "current", "platform"]]
# df_reshaped = pd.pivot_table(df_data,index=["conversation_id", "current"], columns=["root_reply_combined"],aggfunc = np.mean)
gpm = df_data.groupby(["platform", "conversation_id",  "current"]).mean()
gpm

root_reply_combined
platform conversation_id     current                                 
delab    1                   427842656615034887              0.062500
                             458379675751036897              0.250000
                             2766909043494096247             0.083333
                             3337652794504999242             0.500000
                             3417356977311132658             0.071429
...                                                               ...
twitter  1544233528019550208 1544235150040780800             0.062500
                             1544235293896613895             0.055556
                             1544236342376247296             0.050000
                             1544237119673344000             0.045455
                             1544238839069573120             0.041667

[30782 rows x 1 columns]

In [85]:
gpm = gpm.drop(["delab"])
gpm

root_reply_combined
platform conversation_id     current                                 
reddit   661614              26871177                        0.500000
                             29352234                        0.100000
                             33390443                        0.083333
                             36457165                        0.166667
                             91649333                        0.125000
...                                                               ...
twitter  1544233528019550208 1544235150040780800             0.062500
                             1544235293896613895             0.055556
                             1544236342376247296             0.050000
                             1544237119673344000             0.045455
                             1544238839069573120             0.041667

[30765 rows x 1 columns]

In [86]:
# twitter_presences = gpm.loc[["twitter"]]
#reset_index(level=["platform"])
gpm_per_conversation = gpm.groupby(by=["platform", "conversation_id"]).mean()
gpm_per_conversation

root_reply_combined
platform conversation_id                         
reddit   661614                          0.204167
         10955776                        0.531091
         15848916                        0.228333
         17286490                        0.532382
         19851732                        0.606780
...                                           ...
twitter  1544197157187633152             0.451967
         1544203910549733376             0.137267
         1544204096625905664             0.101163
         1544224982275956736             0.037112
         1544233528019550208             0.129300

[890 rows x 1 columns]

In [87]:
gpm_per_platform = gpm.groupby(by=["platform"]).mean()
gpm_per_platform


,root_reply_combined
platform,
reddit,0.525873
twitter,0.206580
